In [58]:
import config
import pandas as pd
from openai import OpenAI

In [13]:
client = OpenAI(api_key=config.OPENAI_API_KEY)


completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "Hi",
        },
    ],
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)


In [16]:
data = {
    "SalePrice": [100000, 120000, 500000],
    "YearBuilt": [2008, 2010, 2020],
    "Area (ft²) ": [2200, 1300, 1500],
    "Address": [
        "789 Oak St, Springfield, 62704",
        "Santa Monica/90401, 123 Ocean Ave",
        "New York, 456 Broadway, 10012",
    ],
}

df = pd.DataFrame(data)
df

,SalePrice,YearBuilt,Area (ft²),Address
0,100000,2008,2200,"789 Oak St, Springfield, 62704"
1,120000,2010,1300,"Santa Monica/90401, 123 Ocean Ave"
2,500000,2020,1500,"New York, 456 Broadway, 10012"


Suggestion for Feature Engineering

In [18]:
prompt = """
I have the following DataFrame for House Price data.

{}

Suggest some new meaningful features I can create from the existing features.
"""

# Format the prompt with the DataFrame
formatted_prompt = prompt.format(df.to_string(index=False))
print(formatted_prompt)


I have the following DataFrame for House Price data.

 SalePrice  YearBuilt  Area (ft²)                            Address
    100000       2008         2200    789 Oak St, Springfield, 62704
    120000       2010         1300 Santa Monica/90401, 123 Ocean Ave
    500000       2020         1500     New York, 456 Broadway, 10012

Suggest some new meaningful features I can create from the existing features.



In [19]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a data scientist that performs feature engineering.",
        },
        {
            "role": "user",
            "content": formatted_prompt
        },
    ],
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Here are some new meaningful features that you can create from the existing features in your DataFrame:\n\n1. `Age of the House`: Calculate the age of the house by subtracting the `YearBuilt` from the current year. This will give you an indication of how old the property is.\n   \n2. `Price per Square Foot`: Calculate the price per square foot by dividing the `SalePrice` by the `Area (ft²)`. This feature can provide insights into the value of the property based on its size.\n\n3. `City`: Extract the city name from the `Address` column. This can be useful for analyzing house prices based on different cities.\n\n4. `Zip Code`: Extract the zip code from the `Address` column. This feature can be used to analyze house prices based on different areas or regions.\n\n5. `Total Rooms`: Create a new feature that estimates the total number of rooms in the property by considering the `Area (ft²)` and average room size.\n\n6. `Is New Construction`: Create a binary fea

In [20]:
print(completion.choices[0].message.content)

Here are some new meaningful features that you can create from the existing features in your DataFrame:

1. `Age of the House`: Calculate the age of the house by subtracting the `YearBuilt` from the current year. This will give you an indication of how old the property is.
   
2. `Price per Square Foot`: Calculate the price per square foot by dividing the `SalePrice` by the `Area (ft²)`. This feature can provide insights into the value of the property based on its size.

3. `City`: Extract the city name from the `Address` column. This can be useful for analyzing house prices based on different cities.

4. `Zip Code`: Extract the zip code from the `Address` column. This feature can be used to analyze house prices based on different areas or regions.

5. `Total Rooms`: Create a new feature that estimates the total number of rooms in the property by considering the `Area (ft²)` and average room size.

6. `Is New Construction`: Create a binary feature that indicates whether the property wa

### Important note: 
Using API in this way can result in very high costs. Dataset will most likely consist of so many tokens. Also, it's costly to make it remember the chat history. I suggest using the web interface instead of api for brainstorming tasks like this. We'll use the API for more trickier tasks!

## Using LLMs for feature generation

In [60]:
data = {
    "SalePrice": [100000, 120000, 500000],
    "YearBuilt": [2008, 2010, 2020],
    "Area (ft²) ": [2200, 1300, 1500],
    "Address": [
        "789 Oak St, Springfield, 62704",
        "Santa Monica/90401, 123 Ocean Ave",
        "New York, 456 Broadway, 10012",
    ],
}

df = pd.DataFrame(data)
df

,SalePrice,YearBuilt,Area (ft²),Address
0,100000,2008,2200,"789 Oak St, Springfield, 62704"
1,120000,2010,1300,"Santa Monica/90401, 123 Ocean Ave"
2,500000,2020,1500,"New York, 456 Broadway, 10012"


In [28]:
# Define a prompt for generating the "State" column
addresses = df["Address"].tolist()
address_string = "\n".join(addresses)
prompt = f"""
I have the following addresses:

{address_string}

Please extract the state for each address and return them as a plain list with no additional text or explanation, in the format ["State1", "State2", "State3"].

"""

In [29]:
print(prompt)


I have the following addresses:

789 Oak St, Springfield, 62704
Santa Monica/90401, 123 Ocean Ave
New York, 456 Broadway, 10012

Please extract the state for each address and return them as a plain list with no additional text or explanation, in the format ["State1", "State2", "State3"].




In [30]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a data scientist that performs feature engineering.",
        },
        {"role": "user", "content": prompt},
    ],
)

print(completion.choices[0].message.content)

["IL", "CA", "NY"]


In [39]:
#Here you might want to implement a smarter parser, that checks if the response is in the correct format.

In [32]:
# Extract and print the response
response_text = completion.choices[0].message.content

# Clean the response to ensure it is in the correct format
import ast

try:
    states = ast.literal_eval(response_text)
except Exception as e:
    print("Error parsing response:", e)
    states = []

# Verify the parsed states
print("Parsed States:")
print(states)
print(type(states))

Parsed States:
['IL', 'CA', 'NY']
<class 'list'>


In [33]:
# Add the "State" column to the DataFrame manually
if len(states) == len(df):
    df["State"] = states
else:
    print("The number of states does not match the number of addresses.")


print(df)

   SalePrice  YearBuilt  Area (ft²)                             Address State
0     100000       2008         2200     789 Oak St, Springfield, 62704    IL
1     120000       2010         1300  Santa Monica/90401, 123 Ocean Ave    CA
2     500000       2020         1500      New York, 456 Broadway, 10012    NY


In [42]:
# Define a prompt for generating the "CloseToSea" column
addresses = df["Address"].tolist()
address_string = "\n".join(addresses)
prompt = f"""
I have the following addresses:

{address_string}

For each address, please indicate whether it is close to the sea or not. Return the results as a list of boolean values in the format [True/False, True/False, True/False] with no additional text and explanation.
"""

In [43]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a data scientist that performs feature engineering.",
        },
        {"role": "user", "content": prompt},
    ],
)

print(completion.choices[0].message.content)

[False, True, False]


In [44]:
response_text = completion.choices[0].message.content

try:
    is_close_to_sea = ast.literal_eval(response_text)
except Exception as e:
    print("Error parsing response:", e)
    is_close_to_sea = []

# Add the "CloseToSea" column to the DataFrame
if len(is_close_to_sea) == len(df):
    df["CloseToSea"] = is_close_to_sea
else:
    print("The number of boolean values does not match the number of addresses.")

In [45]:
df

,SalePrice,YearBuilt,Area (ft²),Address,State,CloseToSea
0,100000,2008,2200,"789 Oak St, Springfield, 62704",IL,False
1,120000,2010,1300,"Santa Monica/90401, 123 Ocean Ave",CA,True
2,500000,2020,1500,"New York, 456 Broadway, 10012",NY,False


## Power of LLMs on Demographic Data

In [46]:
# Create the DataFrame
data = {
    "Name": ["John Smith", "Carlos García", "Marie Dubois", "Hans Schmidt"],
    "Phone Number": [
        "+1 555-123-4567",  # US
        "+34 612 345 678",  # Spain
        "+33 1 23 45 67 89",  # France
        "+49 151 23456789",  # Germany
    ],
}

df = pd.DataFrame(data)
df

,Name,Phone Number
0,John Smith,+1 555-123-4567
1,Carlos García,+34 612 345 678
2,Marie Dubois,+33 1 23 45 67 89
3,Hans Schmidt,+49 151 23456789


In [48]:
# Define a prompt for generating the "Gender" column
# Name -> Gender
names = df["Name"].tolist()
names_string = "\n".join(names)
prompt = f"""
I have the following names:

{names_string}

Please provide the most likely gender (male or female), return the results as a list in the format ["gender", "gender", ...] with no additional text and explanation.
"""

In [49]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a data scientist that performs feature engineering.",
        },
        {"role": "user", "content": prompt},
    ],
)

print(completion.choices[0].message.content)

["male", "male", "female", "male"]


In [51]:
response_text = completion.choices[0].message.content

try:
    genders = ast.literal_eval(response_text)
except Exception as e:
    print("Error parsing response:", e)
    genders = []

# Add the "Gender" column to the DataFrame
if len(genders) == len(df):
    df["Gender"] = genders
else:
    print("The number of genders does not match the number of names.")

In [52]:
df

,Name,Phone Number,Gender
0,John Smith,+1 555-123-4567,male
1,Carlos García,+34 612 345 678,male
2,Marie Dubois,+33 1 23 45 67 89,female
3,Hans Schmidt,+49 151 23456789,male


In [53]:
# Define a prompt for generating the "Country" column
# Phone Number -> Country
phone_numbers = df["Phone Number"].tolist()
phone_numbers_string = "\n".join(phone_numbers)
prompt = f"""
I have the following phone numbers:

{phone_numbers_string}

Please provide the country name from the country code in the phone number. Return the results as a list in the format ["Spain", ...] with no additional text or explanation.
"""

In [54]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a data scientist that performs feature engineering.",
        },
        {"role": "user", "content": prompt},
    ],
)

print(completion.choices[0].message.content)

["United States", "Spain", "France", "Germany"]


In [55]:
response_text = completion.choices[0].message.content

try:
    countries = ast.literal_eval(response_text)
except Exception as e:
    print("Error parsing response:", e)
    countries = []

# Add the "Country" column to the DataFrame
if len(countries) == len(df):
    df["Country"] = countries
else:
    print("The number of countries does not match the number of names.")

In [56]:
df

,Name,Phone Number,Gender,Country
0,John Smith,+1 555-123-4567,male,United States
1,Carlos García,+34 612 345 678,male,Spain
2,Marie Dubois,+33 1 23 45 67 89,female,France
3,Hans Schmidt,+49 151 23456789,male,Germany
